In [33]:
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/k-2x2x2/"

In [34]:
def print_2d_matrix(matrix, decimals=3):
    for row in matrix:
        print(" ".join(f"{val:9.{decimals}f}" for val in row))

In [35]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [36]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# WFN data
wfndata = wfn2py(dirname+'WFN.h5')
wfnqdata = wfn2py(dirname+'WFNq.h5')

# RHO data
rho = inp.read_rho(dirname+"RHO")

# Vxc data
vxc = inp.read_vxc(dirname+"vxc.dat")

In [37]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 8/8 [00:00<00:00, 16929.58it/s]


In [38]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)


        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv, indices_gspace_sorted)
        epsilon.l_epsinv[i_q] = epsinv
        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))

        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")

    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)
calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 8/8 [00:00<00:00, 139.53it/s]

Standard deviation exceeded 1e-16 tolerance: 2.5494915419483335e-15, for i_q:0


In [39]:
from kernel import KernelMtxEl

q0 = [0.001, 0.001, 0.001]
l_qpts = np.array(epsinp.qpts)
l_qpts[0] *= 0
qpts = QPoints.from_cryst(wfndata.kpts.recilat, None, *l_qpts)

kernelclass = KernelMtxEl.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    q0=q0,
    l_epsmats=epsilon.l_epsinv,
    parallel=True,
)

In [40]:
data = kernelclass.kernel_mtxel()
exc = data["exc"]
head = data["head"]
body = data["body"]
wings = data["wings"]

In [41]:
numq = kernelclass.qpts.numq
numk = kernelclass.kpts.numk

In [42]:
ikf, ikc = np.random.randint(0, numk, size=2)
print(f"ikf, ikc = {ikf}, {ikc}")

ikf, ikc = 6, 4


In [43]:
from intkernel import InterpMtxEl

InterpClass = InterpMtxEl.from_BGW(
    wfn_finedata=wfndata,
    wfn_coarsedata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    kernel=kernelclass,
)

In [44]:
coeff_val = InterpClass.coeff_mtxel(ikf, ikf, "val")
coeff_con = InterpClass.coeff_mtxel(ikf, ikc, "con")


print(f"coeff_val is")
print_2d_matrix(coeff_val, decimals=3)

print(f"\n coeff_con is")
print_2d_matrix(coeff_con, decimals=3)

coeff_val is
1.000+0.000j 0.000-0.000j -0.000+0.000j -0.000-0.000j
0.000+0.000j 1.000+0.000j 0.000-0.000j -0.000-0.000j
-0.000-0.000j 0.000+0.000j 1.000-0.000j 0.000+0.000j
-0.000+0.000j -0.000+0.000j 0.000-0.000j 1.000+0.000j

 coeff_con is
0.113+0.490j 0.527+0.104j 0.157+0.294j -0.578+0.114j
0.636-0.299j 0.030+0.025j -0.001+0.024j -0.327-0.631j
-0.501+0.548j 0.029+0.063j -0.023+0.037j 0.472+0.469j
0.003-0.002j 0.527-0.027j -0.570-0.630j -0.001-0.003j


In [45]:
randomarr = np.random.rand(numk, 8)
print(f"\n randomarr is")
print_2d_matrix(randomarr, decimals=3)



 randomarr is
    0.497     0.085     0.552     0.970     0.255     0.021     0.272     0.687
    0.977     0.672     0.503     0.375     0.446     0.803     0.507     0.575
    0.440     0.324     0.166     0.230     0.695     0.266     0.739     0.888
    0.062     0.691     0.151     0.453     0.625     0.932     0.777     0.828
    0.915     0.886     0.031     0.680     0.820     0.339     0.965     0.155
    0.180     0.404     0.073     0.830     0.673     0.301     0.189     0.959
    0.224     0.459     0.792     0.069     0.729     0.070     0.466     0.402
    0.621     0.175     0.279     0.080     0.476     0.831     0.263     0.549


In [46]:
energyval = InterpClass.interp_energy(randomarr, "val")
energycon = InterpClass.interp_energy(randomarr, "con")


print(f"\n energyval is")
print_2d_matrix(energyval, decimals=3)

print(f"\n energycon is")
print_2d_matrix(energycon, decimals=3)


 energyval is
    0.970     0.552     0.085     0.497
    0.375     0.503     0.672     0.977
    0.230     0.166     0.324     0.440
    0.453     0.151     0.691     0.062
    0.680     0.031     0.886     0.915
    0.830     0.073     0.404     0.180
    0.069     0.792     0.459     0.224
    0.080     0.279     0.175     0.621

 energycon is
    0.255     0.021     0.272     0.687
    0.446     0.803     0.507     0.575
    0.695     0.266     0.739     0.888
    0.625     0.932     0.777     0.828
    0.820     0.339     0.965     0.155
    0.673     0.301     0.189     0.959
    0.729     0.070     0.466     0.402
    0.476     0.831     0.263     0.549


In [47]:
is_equal_val = np.allclose(np.flip(randomarr[:, :4], axis = -1), energyval)
is_equal_con = np.allclose(randomarr[:, 4:], energycon)

print(f"Is the interpolated valence energy equal to the random array? {is_equal_val}")
print(f"Is the interpolated conduction energy equal to the random array? {is_equal_con}")

Is the interpolated valence energy equal to the random array? True
Is the interpolated conduction energy equal to the random array? True


In [ ]:
fine_kernel = InterpClass.interp_kernel(head, wings, body, exc)
fine_kernel = fine_kernel[:, :, 0, 0, 0, 0]
print(f"\n fine_kernel is")

print_2d_matrix(fine_kernel, decimals=3)


 fine_kernel is
9627.569-0.000j -0.146+0.555j -0.320+0.108j -0.180-0.323j -0.535-0.818j 0.042+0.145j 0.060-0.237j 0.013+0.002j
-0.146-0.555j 9627.955+0.000j 0.163+0.247j -0.893+0.856j 0.194-0.215j -1.041+0.625j 0.187-0.001j -0.007-0.051j
-0.320-0.108j 0.163-0.247j 9627.955-0.000j 0.222+1.205j -0.074-0.287j -0.008+0.187j 0.667-1.030j 0.002+0.001j
-0.180+0.323j -0.893-0.856j 0.222-1.205j 9627.822+0.000j -0.186+0.014j 0.084+0.019j -0.063-0.059j -0.078+0.107j
-0.535+0.818j 0.194+0.215j -0.074+0.287j -0.186-0.014j 9627.955-0.000j -1.161-0.353j 0.837+0.912j -0.032+0.038j
0.042-0.145j -1.041-0.625j -0.008-0.187j 0.084-0.019j -1.161+0.353j 9627.822+0.000j -0.074-0.044j -0.002+0.004j
0.060+0.237j 0.187+0.001j 0.667+1.030j -0.063+0.059j 0.837-0.912j -0.074+0.044j 9627.822-0.000j 0.018+0.135j
0.013-0.002j -0.007+0.051j 0.002-0.001j -0.078-0.107j -0.032-0.038j -0.002-0.004j 0.018-0.135j 9627.955-0.000j
